# APP 4: Crawl articles

【3C生活】https://www.bnext.com.tw/categories/3c_life/p/1

In [4]:
import requests
import re
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime, timedelta

In [5]:
url_base = 'https://www.bnext.com.tw/categories/3c_life/p/'

In [6]:
all_news = []

for page_no in range(10):
    url_3c = url_base + str(page_no+1)
    req = requests.get(url_3c)
    category_page = BeautifulSoup(req.text, 'lxml')
    category_page.find('div', {'class':"PaginateBox"}).decompose()
    
    articles = category_page.select_one('#app > div.v-application--wrap > div > main > div > div > div > div > div').findAll('a')
    all_news += articles

In [8]:
%%time

category = '3C生活'

categories = []
dates=[]
titles=[]
links=[]
news_ids = []
contents = []
invalid_news = []

serial_no = 1

for j, news_item in enumerate(all_news):

    # Get title
    t = news_item.text
    t = re.sub(' ','', t)
    t = re.sub('\n','@', t)
    t = re.findall('@(.*?)@', t)[0]
    title = t
    print(serial_no, '--', title)

    # Get link
    link = "https://www.bnext.com.tw" + news_item.get('href')
    print(link)
    # Run through all news in current category
    news_page = BeautifulSoup(requests.get( link ).text,'lxml')
    # Get date
    try:
        dt = datetime.strptime(news_page.find('div', {'class':'date'}).text, '%Y.%m.%d')
        date = dt.strftime("%Y-%m-%d")
    except:
        pass
    
    # Generate news ID
    date_str = dt.strftime("%Y%m%d")
    news_id = "3c_" + date_str + "_" + str(serial_no)
    serial_no += 1
    
    # Get content
    try:
        content = news_page.find('div',{'itemprop':"articleBody"})
        cont = content.text
    except AttributeError:
        content = news_page.find('div',{'class':"post_content"})
    # Remove unnecessary content
    try:
        for element in content.findAll('div', {'class': 'pumpkin_middle_text'}):
            element.decompose()
    except:
        pass
    
    
    try:
        for element in content.findAll('h2'):
            element.decompose()
    except:
        pass
    
    try:
        for element in content.findAll('div', {'id': 'pumpkin_159'}):
            element.decompose()
    except:
        pass
    
    try:
        for element in content.findAll('blockquote'):
            element.decompose()
    except:
        pass
    
    try:
        for element in content.findAll('div', {'class': 'inf'}):
            element.decompose()
    except:
        pass
    
    try:
        for element in content.findAll('div', {'class': 'RecommendPost'}):
            element.decompose()
    except:
        pass
    
    try:
        for element in content.findAll('div', {'class': 'LikeUs'}):
            element.decompose()
    except:
        pass
    
    
    try:
        for element in content.findAll('div', {'class': 'tags'}):
            element.decompose()
    except:
        pass
    
    
    try:
        cont = content.text
    except AttributeError:
        print("================= 無法解析:" + category + "第" + str(j+1) + "篇新聞!================")
        invalid_news.append(link)
        serial_no -= 1
        continue
    cont = re.sub('\n','', cont)
    cont = re.sub(' ','', cont)
    cont = re.sub('責任編輯：.*','', cont)
    cont = re.sub('本文授權轉載自：.*','', cont)
    cont = re.sub('延伸閱讀：.*','', cont)
    # Append into lists of data columns
    categories.append(category)
    titles.append(title)
    links.append(link)
    dates.append(date)
    news_ids.append(news_id)
    contents.append(cont)
    # Web scraping interval
    time.sleep(2)

1 -- 三星MicroLED電視首度登台，要價逼近450萬！為何有信心站穩台灣高階市場？
https://www.bnext.com.tw/article/75654/samsung-tv-microled-oled-qled
2 -- VisionPro給蘋果強心針！股價創一年多新高，重回市值3兆美元只差一步
https://www.bnext.com.tw/article/75644/apple-3-trilion
3 -- 「紅魔8Pro」螢幕顯示中國台灣，一查竟揪米客邦違法進口！NCC要罰了
https://www.bnext.com.tw/article/75632/redmagic-8-pro-ncc
4 -- VisionPro實測｜殺手級應用在哪？近視能戴嗎？沉浸感如何？一次解答
https://www.bnext.com.tw/article/75588/vision-pro-real-test
5 -- 華為WatchBuds內藏超迷你耳機、小米手環打Switch拳擊，新款智慧錶亮點一次看
https://www.bnext.com.tw/article/75557/smart-watch-huawei-xiaomi
6 -- 華碩Zenfone10現身，「相機」藏最大亮點？搭載高通最新處理器、AI運算也提升
https://www.bnext.com.tw/article/75605/asus-zenfone-10
7 -- iOS17免費體驗，怎麼做？充電時直接變時鐘、AirDrop進階版可傳大檔影片⋯亮點一次看
https://www.bnext.com.tw/article/75524/apple-ios-17-
8 -- 不讓蘋果獨享掌聲，傳Google、三星和高通今年內推MR設備！安卓陣營怎出招？
https://www.bnext.com.tw/article/75606/samsung-google-me-headset
9 -- 一度打敗蘋果、三星，兩大原因逼華為走下手機神壇！台灣人為何不愛了？
https://www.bnext.com.tw/article/75597/why-huawei-phone-down
10 -- VisionPro能成下個「iPhone時刻」？少了殺手級應用、AI也沒看到？庫克一次解答
http

In [9]:
data = zip(news_ids, dates, titles, categories, contents, links)
df = pd.DataFrame(list(data), columns=['news_id','date','title','category','content','link'])

In [10]:
df.head(6)

,news_id,date,title,category,content,link
0,3c_20230613_1,2023-06-13,三星MicroLED電視首度登台，要價逼近450萬！為何有信心站穩台灣高階市場？,3C生活,三星電子舉辦2023年首場電視發表會，宣布全球市場蟬聯第17年銷量第一，台灣市場銷量在去年也...,https://www.bnext.com.tw/article/75654/samsung...
1,3c_20230613_2,2023-06-13,VisionPro給蘋果強心針！股價創一年多新高，重回市值3兆美元只差一步,3C生活,蘋果公司(AAPL-US)周一(12日)收盤股價創新高，凸顯大型科技股重拾領頭羊的氣勢，該公...,https://www.bnext.com.tw/article/75644/apple-3...
2,3c_20230612_3,2023-06-12,「紅魔8Pro」螢幕顯示中國台灣，一查竟揪米客邦違法進口！NCC要罰了,3C生活,3C代理商米客邦(伊瑪格科技有限公司)自3月於網路募資平台flyingV募資預售「紅魔8Pr...,https://www.bnext.com.tw/article/75632/redmagi...
3,3c_20230609_4,2023-06-09,VisionPro實測｜殺手級應用在哪？近視能戴嗎？沉浸感如何？一次解答,3C生活,這是第一次，我感覺到文字的匱乏與無力。WWDC23第二天，我代表愛范兒，成為了這個世界上最早...,https://www.bnext.com.tw/article/75588/vision-...
4,3c_20230609_5,2023-06-09,華為WatchBuds內藏超迷你耳機、小米手環打Switch拳擊，新款智慧錶亮點一次看,3C生活,"「預計2028年，全球的智慧手表市場規模會達到582.1億美元（約新台幣1兆7,800萬元）...",https://www.bnext.com.tw/article/75557/smart-w...
5,3c_20230609_6,2023-06-09,華碩Zenfone10現身，「相機」藏最大亮點？搭載高通最新處理器、AI運算也提升,3C生活,華碩最新5G旗艦手機Zenfone10現身了！華碩宣布，6月29日晚間9點將線上發表最新5G...,https://www.bnext.com.tw/article/75605/asus-ze...


In [11]:
df.shape

(190, 6)

In [12]:
df.to_csv("3c_news_190.csv", sep="|", index=False)

## Read Data

In [13]:
df2 = pd.read_csv('3c_news_190.csv', sep='|')
df2.head(6)

,news_id,date,title,category,content,link
0,3c_20230613_1,2023-06-13,三星MicroLED電視首度登台，要價逼近450萬！為何有信心站穩台灣高階市場？,3C生活,三星電子舉辦2023年首場電視發表會，宣布全球市場蟬聯第17年銷量第一，台灣市場銷量在去年也...,https://www.bnext.com.tw/article/75654/samsung...
1,3c_20230613_2,2023-06-13,VisionPro給蘋果強心針！股價創一年多新高，重回市值3兆美元只差一步,3C生活,蘋果公司(AAPL-US)周一(12日)收盤股價創新高，凸顯大型科技股重拾領頭羊的氣勢，該公...,https://www.bnext.com.tw/article/75644/apple-3...
2,3c_20230612_3,2023-06-12,「紅魔8Pro」螢幕顯示中國台灣，一查竟揪米客邦違法進口！NCC要罰了,3C生活,3C代理商米客邦(伊瑪格科技有限公司)自3月於網路募資平台flyingV募資預售「紅魔8Pr...,https://www.bnext.com.tw/article/75632/redmagi...
3,3c_20230609_4,2023-06-09,VisionPro實測｜殺手級應用在哪？近視能戴嗎？沉浸感如何？一次解答,3C生活,這是第一次，我感覺到文字的匱乏與無力。WWDC23第二天，我代表愛范兒，成為了這個世界上最早...,https://www.bnext.com.tw/article/75588/vision-...
4,3c_20230609_5,2023-06-09,華為WatchBuds內藏超迷你耳機、小米手環打Switch拳擊，新款智慧錶亮點一次看,3C生活,"「預計2028年，全球的智慧手表市場規模會達到582.1億美元（約新台幣1兆7,800萬元）...",https://www.bnext.com.tw/article/75557/smart-w...
5,3c_20230609_6,2023-06-09,華碩Zenfone10現身，「相機」藏最大亮點？搭載高通最新處理器、AI運算也提升,3C生活,華碩最新5G旗艦手機Zenfone10現身了！華碩宣布，6月29日晚間9點將線上發表最新5G...,https://www.bnext.com.tw/article/75605/asus-ze...
